In [1]:
# encoding: utf-8
import configparser
config = configparser.ConfigParser()
config['mariadb'] = {}

# それぞれ値を入力する
config['mariadb']['username'] = 'root'
config['mariadb']['password'] = '8589'
config['mariadb']['database'] = 'data_analysis'

with open('config.ini', 'w') as configfile:
    config.write(configfile)

In [2]:
# encoding: utf-8
from lxml import etree
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import glob
import sqlalchemy as sa
import sqlalchemy.ext.declarative
import pymysql
import configparser

In [3]:
tree = etree.parse('grants_masterxml_kaken/institution_master_kakenhi.xml')

institutionlist = []
for institution_table in tree.iterfind("institution_table"):
    for institution in institution_table.iterfind("institution"):
        institutionname = institution.find("name[@lang='ja']").text
        for code in institution.iterfind("code"):
            row = [
                code.get("type"),
                code.text,
                institutionname,
            ]
            institutionlist.append(row)
            
df = pd.DataFrame(institutionlist)
df.columns = ['code_type', 'code_number', 'institution_name']
df

,code_type,code_number,institution_name
0,nii,0010101,北海道大学
1,mext,10101,北海道大学
2,jsps,10101,北海道大学
3,nii,0010102,北海道教育大学
4,mext,10102,北海道教育大学
5,jsps,10102,北海道教育大学
6,nii,0010103,室蘭工業大学
7,mext,10103,室蘭工業大学
8,jsps,10103,室蘭工業大学
9,nii,0010104,小樽商科大学


In [4]:
df.duplicated().any()

True

In [5]:
df = df.drop_duplicates()
df

,code_type,code_number,institution_name
0,nii,0010101,北海道大学
1,mext,10101,北海道大学
2,jsps,10101,北海道大学
3,nii,0010102,北海道教育大学
4,mext,10102,北海道教育大学
5,jsps,10102,北海道教育大学
6,nii,0010103,室蘭工業大学
7,mext,10103,室蘭工業大学
8,jsps,10103,室蘭工業大学
9,nii,0010104,小樽商科大学


In [6]:
Base = sa.ext.declarative.declarative_base()

class Institution(Base):
    __tablename__ = 'kaken_master_institution'
    id = sa.Column(sa.Integer, primary_key=True)
    code_type = sa.Column(sa.String(256))
    code_number = sa.Column(sa.Integer)
    institution_name = sa.Column(sa.String(256))

In [7]:
config = configparser.ConfigParser()
config.read('config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

In [8]:
engine = sa.create_engine(url, echo=True)
Base.metadata.create_all(engine)
df.to_sql('kaken_master_institution', engine, if_exists='replace')

2018-05-17 20:45:03,163 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2018-05-17 20:45:03,178 INFO sqlalchemy.engine.base.Engine {}
2018-05-17 20:45:03,178 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2018-05-17 20:45:03,178 INFO sqlalchemy.engine.base.Engine {}
2018-05-17 20:45:03,194 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2018-05-17 20:45:03,194 INFO sqlalchemy.engine.base.Engine {}
2018-05-17 20:45:03,194 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2018-05-17 20:45:03,194 INFO sqlalchemy.engine.base.Engine {}
2018-05-17 20:45:03,210 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2018-05-17 20:45:03,210 INFO sqlalchemy.engine.base.Engine {}
2018-05-17 20:45:03,210 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2018-05-17 20

In [9]:
engine = sa.create_engine(url, echo=True)
Base.metadata.create_all(engine)
df.to_sql('kaken_master_institution', engine, if_exists='replace')

2018-05-17 20:45:10,716 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2018-05-17 20:45:10,716 INFO sqlalchemy.engine.base.Engine {}
2018-05-17 20:45:10,716 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2018-05-17 20:45:10,716 INFO sqlalchemy.engine.base.Engine {}
2018-05-17 20:45:10,732 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2018-05-17 20:45:10,732 INFO sqlalchemy.engine.base.Engine {}
2018-05-17 20:45:10,732 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2018-05-17 20:45:10,732 INFO sqlalchemy.engine.base.Engine {}
2018-05-17 20:45:10,732 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2018-05-17 20:45:10,747 INFO sqlalchemy.engine.base.Engine {}
2018-05-17 20:45:10,747 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2018-05-17 20